In [94]:
import sys
import os
import numpy as np
import pandas as pd
import tweedie
import warnings
warnings.filterwarnings("ignore")

from statsmodels.genmod.families.links import Log
from patsy import dmatrix

sys.path.append(os.path.join(sys.path[0], ".."))

from mcglm import MCGLM, mc_id

This notebook conducts some simulations to prove the overall convergence of the library. 

##### 1 - Tweedie convergence

This section presents a simulation study which validates Tweedie regressions. It produces a outcome vector $Y$ as indepedent Tweedie realizations, with the vector $\mu$ as the mean parameters and a constant dispersion parameter $\phi$. To establish a mathematical notation, $Y_i \sim Tw_p(\mu_i, \phi)$ and $g(\mu_i) = \eta_i = x_i\beta$. Moreover, we set three parameters(regression): $\beta_0$, $\beta_1$ and $\beta_2$ defined as (2, 0.8, -1.5), and two covariates: a sequence within -1 to 1 and a categorical two-level randomly chosen. The vector $\mu$ is calculated by means of the linear operation between covariates and regression parameters. Finally, the parameter `power` is vital to emulate the distribution as some EDM - Exponential Dispersion Family; we benchmark thre values for power (1.1, 1.5, 1.9).

By some Tweedie simulations, with the aid of the package {tweedie}, we aim to validate whether the scale parameter from tweedie simulations and the scale from the model match. 

The method below helps us to craft simulation by parameters `n_sim` as number of simulations; `sample_size` as sample size; `dispersion` as the scale parameter; `power` as the power parameter. 

In [111]:
def generate_tweedie_simulation(n_sim: int, sample_size: int, dispersion: float, power: float):
    historical_scale_from_simulation = []
    for _ in range(n_sim):
        cov1 = np.arange(-1.0, 1.0, 1/(sample_size/2))  #np.random.uniform(-1, 1, SAMPLE_SIZE)
        cov2 = np.random.choice([0, 1], size=sample_size)

        beta_0 = 2
        beta_1 = 0.8
        beta_2 = -1.5

        eta = beta_0 + beta_1*cov1 + beta_2*cov2
        mu = Log().inverse(eta)

        y = tweedie.tweedie(mu=mu, p=power, phi=dispersion).rvs(sample_size)

        data = pd.DataFrame({'y': y, 'cov1': cov1, 'cov2': cov2})

        data['cov2'] = data['cov2'].astype('str')
        X = dmatrix("~ cov1 + cov2", data, return_type="dataframe")

        # Z specification
        Z = [mc_id(data)] 

        # Model fitting
        mcglm = MCGLM(endog=data['y'], exog=X, z=Z, link='log', variance='tweedie', power=power, power_fixed=True)

        mcglmresults = mcglm.fit()

        historical_scale_from_simulation.append(mcglmresults._dispersion[0]['scalelist'][0])
        
    return historical_scale_from_simulation

Simulation for power set as `1.01`.

In [116]:
simulation_1 = {'n_sim':20, 'sample_size':300, 'dispersion':1.5, 'power':1.1}
scale_simulation_1 = generate_tweedie_simulation(**simulation_1)

simulation_2 = {'n_sim':20, 'sample_size':300, 'dispersion':15, 'power':1.1}
scale_simulation_2 = generate_tweedie_simulation(**simulation_2)

simulation_3 = {'n_sim':20, 'sample_size':300, 'dispersion':40, 'power':1.1}
scale_simulation_3 = generate_tweedie_simulation(**simulation_3)

In [117]:
scale_simulation_1

[1.49,
 1.354,
 1.558,
 1.416,
 1.472,
 1.25,
 1.556,
 1.605,
 1.747,
 1.367,
 1.641,
 1.544,
 1.502,
 1.545,
 1.708,
 1.569,
 1.448,
 1.519,
 1.606,
 1.3]

In [88]:
scale_simulation_dispersion_02_sample_100 = generate_tweedie_simulation(n_sim=100, sample_size=100, dispersion=0.2)
scale_simulation_dispersion_02_sample_200 = generate_tweedie_simulation(n_sim=100, sample_size=200, dispersion=0.2)
scale_simulation_dispersion_02_sample_400 = generate_tweedie_simulation(n_sim=100, sample_size=400, dispersion=0.2)
scale_simulation_dispersion_02_sample_800 = generate_tweedie_simulation(n_sim=100, sample_size=800, dispersion=0.2)

/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: divide by zero encountered in log
  endog * np.log(endog / mu) + (mu - endog))
/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: invalid value encountered in multiply
  endog * np.log(endog / mu) + (mu - endog))
/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: divide by zero encountered in log
  endog * np.log(endog / mu) + (mu - endog))
/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: invalid value encountered in multiply
  endog * np.log(endog / mu) + (mu - endog))
/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: divide by zero encountered

In [90]:
scale_simulation_dispersion_2_sample_100 = generate_tweedie_simulation(n_sim=100, sample_size=100, dispersion=2)
scale_simulation_dispersion_2_sample_200 = generate_tweedie_simulation(n_sim=100, sample_size=200, dispersion=2)
scale_simulation_dispersion_2_sample_400 = generate_tweedie_simulation(n_sim=100, sample_size=400, dispersion=2)
scale_simulation_dispersion_2_sample_800 = generate_tweedie_simulation(n_sim=100, sample_size=800, dispersion=2)

/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: divide by zero encountered in log
  endog * np.log(endog / mu) + (mu - endog))
/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: invalid value encountered in multiply
  endog * np.log(endog / mu) + (mu - endog))
/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: divide by zero encountered in log
  endog * np.log(endog / mu) + (mu - endog))
/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: invalid value encountered in multiply
  endog * np.log(endog / mu) + (mu - endog))
/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: divide by zero encountered

In [91]:
scale_simulation_dispersion_5_sample_100 = generate_tweedie_simulation(n_sim=100, sample_size=100, dispersion=5)
scale_simulation_dispersion_5_sample_200 = generate_tweedie_simulation(n_sim=100, sample_size=200, dispersion=5)
scale_simulation_dispersion_5_sample_400 = generate_tweedie_simulation(n_sim=100, sample_size=400, dispersion=5)
scale_simulation_dispersion_5_sample_800 = generate_tweedie_simulation(n_sim=100, sample_size=800, dispersion=5)

/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: divide by zero encountered in log
  endog * np.log(endog / mu) + (mu - endog))
/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: invalid value encountered in multiply
  endog * np.log(endog / mu) + (mu - endog))
/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: divide by zero encountered in log
  endog * np.log(endog / mu) + (mu - endog))
/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: invalid value encountered in multiply
  endog * np.log(endog / mu) + (mu - endog))
/home/jean/dev/Environments/master_thesis/lib/python3.9/site-packages/statsmodels/genmod/families/family.py:1582: RuntimeWarning: divide by zero encountered

In [92]:
scale_simulation_dispersion_5_sample_800

[4.928,
 5.538,
 5.421,
 4.418,
 5.074,
 5.473,
 5.355,
 4.494,
 5.066,
 4.627,
 4.958,
 5.639,
 5.172,
 4.751,
 4.294,
 5.411,
 4.685,
 4.849,
 5.015,
 5.136,
 5.172,
 5.229,
 5.99,
 4.962,
 4.669,
 4.853,
 4.449,
 5.584,
 5.129,
 5.222,
 4.982,
 5.035,
 5.141,
 4.806,
 5.375,
 4.452,
 5.223,
 4.337,
 4.933,
 4.519,
 4.859,
 5.033,
 4.53,
 5.487,
 5.709,
 4.763,
 5.151,
 5.482,
 5.047,
 4.678,
 5.326,
 4.733,
 5.678,
 4.724,
 5.575,
 4.344,
 5.883,
 5.327,
 4.426,
 5.266,
 4.854,
 5.239,
 4.834,
 4.949,
 5.344,
 4.972,
 5.692,
 4.868,
 4.697,
 5.206,
 5.088,
 4.595,
 5.111,
 4.428,
 5.432,
 5.175,
 5.193,
 5.63,
 5.092,
 5.099,
 4.556,
 4.95,
 5.148,
 5.525,
 4.709,
 5.286,
 4.502,
 5.037,
 4.796,
 4.768,
 4.82,
 4.728,
 5.116,
 5.229,
 5.286,
 4.937,
 4.795,
 5.195,
 4.914,
 5.111]